In [ ]:
from sklearn.utils import resample
import matplotlib.pyplot as plt
import math
from glob import glob
import seaborn as sns
from PIL import Image,ImageOps
from cv2 import imread, resize,cvtColor, imwrite
from os.path import exists
from scipy import ndimage
from skimage.measure import label, regionprops
import warnings
warnings.filterwarnings("ignore")
 
from IPython.display import display
import os

import cv2
 
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import confusion_matrix, accuracy_score
 
from tensorflow.keras.utils import to_categorical
import itertools
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

 
import numpy as np, pandas as pd
 
 
 

In [ ]:
base_skin_dir = os.path.join('..', 'input')

# Merging images from both folders HAM10000_images_part1.zip and HAM10000_images_part2.zip into one dictionary

imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir,"skin-cancer-mnist-ham10000/", '*', '*.jpg'))}
lesion_type_dict = {
    'nv': 'Melanocytic nevi (nv)',
    'mel': 'Melanoma (mel)',
    'bkl': 'Benign keratosis-like lesions (bkl)',
    'bcc': 'Basal cell carcinoma (bcc)',
    'akiec': 'Actinic keratoses (akiec)',
    'vasc': 'Vascular lesions (vasc)',
    'df': 'Dermatofibroma (df)'
}
label_mapping = {
    0: 'nv',
    1: 'mel',
    2: 'bkl',
    3: 'bcc',
    4: 'akiec',
    5: 'vasc',
    6: 'df'
}
reverse_label_mapping = dict((value, key) for key, value in label_mapping.items())

In [ ]:
paths=[]
ids=[]
for dirname, _, filenames in os.walk('/kaggle/input/ham10000-lesion-segmentations/HAM10000_segmentations_lesion_tschandl'):
    for filename in filenames:
        ids.append(filename.split("_seg")[0])
        paths.append(os.path.join(dirname, filename))

In [ ]:
def get_segment(img_id):
    contains_abc = segment_df['image_id'].str.contains(img_id)
    return segment_df[contains_abc].segment_path.values[0]

In [ ]:
df_skin= pd.read_csv("/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv")
df_skin.head()



    
df_skin['path'] = df_skin['image_id'].map(imageid_path_dict.get)
df_skin['cell_type'] = df_skin['dx'].map(lesion_type_dict.get) 
df_skin['cell_type_idx'] = pd.Categorical(df_skin['cell_type']).codes
df_skin.sample(5)

In [ ]:
 df_skin['image_id'].map(imageid_path_dict.get)

In [ ]:
import numpy as np
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans

def quantize_channels(x, k):
    quantized_x = x.copy()
    for d in range(3):
        channel = x[:, :, d].copy()
        # k_means = MiniBatchKMeans(k, compute_labels=False, n_init='auto')
        k_means = KMeans(n_clusters=k, random_state=0, n_init='auto')
        k_means.fit(channel.reshape(-1, 1))
        labels = k_means.predict(channel.reshape(-1, 1))
        quantized_x[:,:,d] = labels.reshape(channel.shape)
    return quantized_x

def imquantize(x, levels):
    quantized_img  = quantize_channels(x, levels)
    M, N, _ = x.shape
    Index = np.zeros((M,N)).astype('uint8')
    for i in range(M):
        for j in range(N):
            # (0 0 0)base8 = (0*8^2 + 0*8^1 + 0*8^0)base10
            Index[i][j] = quantized_img[i][j][0] * 64 + quantized_img[i][j][1] * 8 + quantized_img[i][j][2]
            # Index[i][j] = quantized_img[i][j][0] * (levels*levels) + quantized_img[i][j][1] * levels + quantized_img[i][j][2]
    return Index


from scipy.stats import gamma, entropy
import numpy as np
from skimage.filters import threshold_otsu

def fuzzy_divergence_thresholding(gray):
    with np.errstate(invalid='ignore'):
        # Compute the Otsu threshold as the initial guess
        otsu_thresh = threshold_otsu(gray)

        # Define the range of candidate threshold values
        thresh_range = np.arange(otsu_thresh - 0.1, otsu_thresh + 0.1, 0.01)

        # Compute the fuzzy histogram
        fuzzy_hist = np.zeros((256, 2))
        for i in range(256):
            # Use gamma distribution to compute the membership function
            # fuzzy_hist[i][0] = gamma.pdf(i, gray.flat[i], scale=20)
            fuzzy_hist[i][0] = np.exp(-((i - gray.flat[i]) / 20) ** 2)
            fuzzy_hist[i][1] = 1 - fuzzy_hist[i][0]

        # Compute the divergence between the fuzzy histograms
        divergence = np.zeros(len(thresh_range))
        for i, thresh in enumerate(thresh_range):
            foreground = fuzzy_hist[:int(thresh * 255), 0].sum() + fuzzy_hist[int(thresh * 255):, 1].sum()
            background = fuzzy_hist[:int(thresh * 255), 1].sum() + fuzzy_hist[int(thresh * 255):, 0].sum()
            divergence[i] = foreground * np.log(foreground / (foreground + background)) + background * np.log(background / (foreground + background))

        # Find the threshold value that minimizes the divergence
        # min_div_idx = (np.argmin(divergence)+np.argmax(-divergence))//2
        min_div_idx = np.argmin(divergence)
        threshold = thresh_range[min_div_idx]

    return threshold


import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from skimage import color
from skimage import exposure, img_as_float
from skimage.filters import threshold_otsu
import cv2
 
from sklearn.preprocessing import MinMaxScaler
 
from sklearn.cluster import KMeans


def segmentation(img):
    n = 0.5; #distance scaling parameter (n) such that 0.1 ≤ n ≤ 1.0
    NIC = 3; # number of image components
    levels = 8; # image quantization level
    NCD = 512; #levels^3; # maximum number of desired colors
    LCL = 0; # lab color light
    LCA = 1; # lab color A
    LCB = 2; # lab color B
    PXC = 3; # pixel x-coordinate
    PYC = 4; # pixel y-coordinate
    PDC = 5; # pixel distance to image center
    CPC = 6; # cluster pixel count
    CPL = 7; # cluster pixel label
    CCC = 7; # cluster color contrast
    CSS = 8; # cluster saliency score

    K = 0

    img_arr = np.array(img)

    #---- Image Quantization ----
    M, N, _ = img_arr.shape

    Index = imquantize(img_arr, levels)

    # Convert image to LAB color space
    lab_image = color.rgb2lab(img)

    Input = exposure.rescale_intensity(lab_image, out_range=(0, 1))

    Palette = np.zeros((NCD,9))

    for x in range(M):
        for y in range(N):
            Palette[Index[x, y], CPC] += 1
            Palette[Index[x, y], LCL] += Input[x, y, LCL]
            Palette[Index[x, y], LCA] += Input[x, y, LCA]
            Palette[Index[x, y], LCB] += Input[x, y, LCB]
            Palette[Index[x, y], PXC] += x
            Palette[Index[x, y], PYC] += y

    Cluster = np.zeros((NCD,9))
    for z in range(NCD):
        if Palette[z, CPC] > 0:
            Palette[z, CPL] = K
            Palette[z, PXC] /= M
            Palette[z, PYC] /= N
            Palette[z, 0:PYC+1] /= Palette[z, CPC]
            Cluster[K, 0:CPC+1] = Palette[z, 0:CPC+1]
            K += 1

    Cluster = np.delete(Cluster, range(K, NCD), axis=0)
    Wr = Cluster[:, CPC] / (M * N)

    for x in range(K):
        Cluster[x, CCC] = 0
        for y in range(K):
            Cluster[x, CCC] += Wr[y] * np.linalg.norm(Cluster[x, :NIC] - Cluster[y, :NIC])

    for x in range(M):
        for y in range(N):
            cluster_index = int(Palette[Index[x, y], CPL])
            Cluster[cluster_index, PDC] += ((x / M - 0.5) ** 2) + ((y / N - 0.5) ** 2)

    for z in range(K):
        Cluster[z, PDC] = Cluster[z, PDC]/(n * n * Cluster[z, CPC])

    for x in range(K):
        Cluster[x, CSS] = 0
        for y in range(K):
            Ds = np.linalg.norm(Cluster[x, PXC:PYC] - Cluster[y, PXC:PYC])
            Phixy = (Cluster[x, CCC] + 0.05) / (Cluster[y, CCC] + 0.05)
            Cluster[x, CSS] += Wr[y] * Phixy * np.exp(-Ds)
        
        Cluster[x, CSS] = np.exp(-Cluster[x, PDC]) * (Wr[x] * Cluster[x, CCC] + Cluster[x, CSS])


    # Cluster[:, CSS] = minmax_scale(Cluster[:, CSS])
    Cluster[:, CSS] = MinMaxScaler().fit_transform(np.array(Cluster[:, CSS]).reshape(-1, 1)).ravel()

    for x in range(M):
        for y in range(N):
            Input[x, y, LCL] = Cluster[int(Palette[Index[x, y], CPL]), CSS]

    # Apply Fuzzy Divergence thresholding to Input
    # threshold = threshold_otsu(Input[:, :, LCL])
    threshold = abs(fuzzy_divergence_thresholding(Input[:, :, LCL]))
    Output = Input[:, :, LCL] >= threshold

    Output = Output.astype(np.uint8) * 255


   # assuming that the variable "Output" contains the binary image
    Output = cv2.morphologyEx(Output, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)))

    # Perform connected component labeling and analysis
    totalLabels, labels, stats, centroids = cv2.connectedComponentsWithStats(Output, 4, cv2.CV_32S)

    # Find the center of the image
    h, w = Output.shape
    cx = w // 2
    cy = h // 2

    # Compute the distance of each pixel from the center
    dist = np.sqrt((np.arange(w) - cx) ** 2 + (np.arange(h)[:, None] - cy) ** 2)

    # Find the largest component area
    max_area = 0
    for i in range(1, totalLabels):
        area = stats[i][cv2.CC_STAT_AREA]
        if area > max_area:
            max_area = area
            x, y = centroids[i]
    
    min_size = 0.2 * max_area
    max_dist = dist[int(y), int(x)]

    for i in range(1, totalLabels):
        area = stats[i][cv2.CC_STAT_AREA]
        x, y = centroids[i]
        # Compute the distance of the centroid from the center
        d = dist[int(y), int(x)]
        # If the area is smaller than min_size and the distance is larger than max_dist, remove the component
        if area < min_size and d > max_dist:
            Output[labels == i] = 0

    # # Convert binary image into an array of (x,y) coordinates
    # coords = np.column_stack(np.nonzero(Output))

    # # Perform k-means clustering on the coordinates
    # kmeans = KMeans(n_clusters=2)
    # kmeans.fit(coords)

    # # Get the cluster labels for each coordinate
    # labels = kmeans.labels_

    # # Find the cluster with the maximum number of components
    # largest_cluster = max(set(labels), key=list(labels).count)

    # # Create an output image with the same shape as the input image
    # out_im = np.zeros_like(Output)

    # # Assign each pixel in the output image to the largest cluster based on its label
    # for i, coord in enumerate(coords):
    #     if labels[i] == largest_cluster:
    #         out_im[coord[0], coord[1]] = 255
    
    # Output = out_im

    return Output


# #  Load image
# img = Image.open("D:\\archive\HAM10000_images_part_1\ISIC_0024481.jpg")
# output = segmentation(img)
# output = Image.fromarray(output.astype('uint8'))
# output.show()


import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import threshold_otsu
from skimage.segmentation import mark_boundaries
from skimage.color import rgb2hsv
from skimage.filters import gaussian


def blackhat(img):
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,20))
    hat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, rectKernel)
    return hat

def tophat(img):
    rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,20))
    hat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, rectKernel)
    return hat

def findThreshold(img):
    fig, ax = plt.subplots(1, 1)
    y,x,_ = ax.hist(img.ravel(), bins=256, range=[0, 255])
    x_index = x[np.argmax(y)]
    y_mean = y.mean()
    index = int(x_index)
    for i in range(index, 256):
        if(y_mean > y[np.where(x==x[i])]):
            index = i
            break
#     ax.axvline(x[index], color='k', linestyle='dashed', linewidth=1)
#     plt.show()
    return index

# masking
def connected_component_label(img):
    connectivity = 4
    numLabels, labels, stats, _ = cv2.connectedComponentsWithStats(img, connectivity, cv2.CV_32S)
    area = np.mean(stats[1:, cv2.CC_STAT_AREA])
    mask = np.zeros(img.shape, dtype="uint8")
    for i in range(1, numLabels):
        if stats[i, cv2.CC_STAT_AREA] > area:
            mask[labels == i] = 255
    maskarea = np.sum(mask) / 255
    return maskarea, mask

def removeNestedContours(binary):
    # Find the contours in the binary image
    contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the indices of the contours that do not have a parent (i.e. they are not nested)
    unique_indices = np.where(hierarchy[0][:, 3] == -1)[0]

    # Get the unique contours from the indices
    unique_contours = [contours[i] for i in unique_indices]

    # creating white mask
    mask = np.zeros_like(binary)
    # set all pixels within contours to 255
    cv2.drawContours(mask, unique_contours, -1, 255, -1)
    return mask

def testHoughLines(img):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    # Apply Hough Line Transform to detect hairlines
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=20, minLineLength=25, maxLineGap=5)
    if lines is not None:
        return len(lines)
    else:
        return 0

def process(original_path,filename):#, fldrpath, filename):  # path = filename

    original = cv2.imread(original_path)

    # converting to LAB color space
    lab = cv2.cvtColor(original, cv2.COLOR_BGR2LAB)
    l_channel, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    cl = clahe.apply(l_channel)
    limg = cv2.merge((cl, a, b))

    # converting back to original
    enhanced_img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)

    enhanced_r = enhanced_img[:, :, 2]
    enhanced_b = enhanced_img[:, :, 0]

    black = blackhat(enhanced_r)
    ret, threshB = cv2.threshold(black, findThreshold(black), 255, cv2.THRESH_BINARY)

    # filtering
    hairareaB, filteredB = connected_component_label(threshB)

    # inpaint the original image depending on the mask
    resultB = cv2.inpaint(original, filteredB, 1, cv2.INPAINT_TELEA)

    # TOPHAT
    top = tophat(enhanced_b)
    ret, threshT = cv2.threshold(top, findThreshold(top), 255, cv2.THRESH_BINARY)

    # filtering
    hairareaT, filteredT = connected_component_label(threshT)

    # inpaint the original image depending on the mask
    resultT = cv2.inpaint(original, filteredT, 1, cv2.INPAINT_TELEA)

    # segmentation
    binary_image1 = segmentation(resultB)
    binary_image2 = segmentation(resultT)

    binary_image1 = removeNestedContours(binary_image1)
#     cv2.imwrite(f'{fldrpath}\{filename} Mask B.png', binary_image1)
    binary_image2 = removeNestedContours(binary_image2)
#     cv2.imwrite(f'{fldrpath}\{filename} Mask T.png', binary_image2)
#     return binary_image1,binary_image2

    white_pixels1 = cv2.countNonZero(binary_image1)
    white_pixels2 = cv2.countNonZero(binary_image2)

    if(white_pixels1 < white_pixels2):
        minmask = binary_image1
    else:
        minmask = binary_image2

    from pathlib import Path
    Path('/kaggle/working/mask_image').mkdir(parents=True, exist_ok=True)
#     Path('/kaggle/working/clear_image').mkdir(parents=True, exist_ok=True)
#     Path('/kaggle/working/mask_image').mkdir(parents=True, exist_ok=True)
    
#     cv2.imwrite(f' Mask_{fldrpath}\{filename}.png', binary_image2)
    if testHoughLines(original) > 50:
        if testHoughLines(resultB) < testHoughLines(resultT):
#             print("a")
            masked_img = cv2.bitwise_and(resultB, resultB, mask=minmask)
            cv2.imwrite(f'/kaggle/working/mask_image/{filename}.png', minmask)
#             cv2.imwrite(f'/kaggle/working/clear_image/{filename}.png', resultB)
#             cv2.imwrite(f'/kaggle/working/binary_mask/{filename}.png', masked_img)
            
#             return masked_img,minmask,resultB
        else:
#             print("b")
            masked_img = cv2.bitwise_and(resultT, resultT, mask=minmask)
            cv2.imwrite(f'/kaggle/working/mask_image/{filename}.png', minmask)
#             cv2.imwrite(f'/kaggle/working/clear_image/{filename}.png', resultT)
#             cv2.imwrite(f'/kaggle/working/binary_mask/{filename}.png', masked_img)
            
#             return masked_img,minmask,resultT
    else:
#         print("c")
        masked_img = cv2.bitwise_and(original, original, mask=minmask)
        cv2.imwrite(f'/kaggle/working/mask_image/{filename}.png', minmask)
#         cv2.imwrite(f'/kaggle/working/clear_image/{filename}.png', resultT)
#         cv2.imwrite(f'/kaggle/working/binary_mask/{filename}.png', masked_img)
#         return masked_img,minmask,original

    


In [ ]:
# df=df_skin.sample(2)
paths=df_skin[8000:].path.values
image_ids=df_skin[8000:].image_id.values


del df_skin

len(paths)




In [ ]:
for index, image_path in tqdm(enumerate(paths)):
#     print(f"Image ID: {image_id}, Image Path: {image_path}")
    process(image_path,image_ids[index])